In [1]:
import pandas as pd
from pandasql import sqldf

In [2]:
# Célula 2: Carregando os arquivos de dados para a memória

# O caminho '../input/brazilian-ecommerce/' é o padrão do Kaggle para este dataset
caminho_dados = '../input/brazilian-ecommerce/'

print("Carregando arquivos, isso pode levar um momento...")

# Criando as variáveis (que funcionarão como nossas 'tabelas' para o SQL)
customers = pd.read_csv(caminho_dados + 'olist_customers_dataset.csv')
orders = pd.read_csv(caminho_dados + 'olist_orders_dataset.csv')
order_items = pd.read_csv(caminho_dados + 'olist_order_items_dataset.csv')
products = pd.read_csv(caminho_dados + 'olist_products_dataset.csv')
payments = pd.read_csv(caminho_dados + 'olist_order_payments_dataset.csv')
sellers = pd.read_csv(caminho_dados + 'olist_sellers_dataset.csv')

print("Dados carregados com sucesso!")
print("As 'tabelas' customers, orders, etc., agora existem e estão prontas para serem usadas.")

Carregando arquivos, isso pode levar um momento...
Dados carregados com sucesso!
As 'tabelas' customers, orders, etc., agora existem e estão prontas para serem usadas.


### Análise Inicial: Qual a Distribuição Geográfica dos Nossos Clentes?

A primeira etapa para entender o negócio é saber onde nossos clientes estão localizados. Esta análise inicial identifica os estados com a maior concentração de clientes, utilizando apenas a tabela *customers*.

In [3]:
query_clientes_por_estado = """
    SELECT
        customer_state AS estado,
        COUNT(customer_unique_id) AS total_de_clientes
    FROM
        customers
    GROUP BY
        customer_state
    ORDER BY
        total_de_clientes DESC;
"""

# Executando a consulta
clientes_por_estado = sqldf(query_clientes_por_estado)

# Exibindo o resultado
print("Distribuição de Clientes por Estado")
display(clientes_por_estado)

Distribuição de Clientes por Estado


,estado,total_de_clientes
0,SP,41746
1,RJ,12852
2,MG,11635
3,RS,5466
4,PR,5045
5,SC,3637
6,BA,3380
7,DF,2140
8,ES,2033
9,GO,2020


### Insight 2: As Categorias de Produtos Mais Vendidas
Nesta análise, vamos identificar as 10 categorias de produtos com maior volume de vendas para entender o foco principal do negócio. Para isso, precisaremos conectar os dados de itens vendidos (`order_items`) com os dados de produtos (`products`).

In [4]:
query_top_categorias = """
SELECT 
    p.product_category_name AS categoria,
    COUNT(oi.product_id) AS total_vendido
FROM
    order_items oi
JOIN
    products p ON oi.product_id = p.product_id
WHERE
    p.product_category_name IS NOT NULL
GROUP BY
    p.product_category_name
ORDER BY
    total_vendido DESC
LIMIT 10;
"""

# Executando a consulta
top_categorias = sqldf(query_top_categorias)

# Exibindo o resultado
print("Top 10 Categorias de Produtos Mais Vendidas")
display(top_categorias)

Top 10 Categorias de Produtos Mais Vendidas


,categoria,total_vendido
0,cama_mesa_banho,11115
1,beleza_saude,9670
2,esporte_lazer,8641
3,moveis_decoracao,8334
4,informatica_acessorios,7827
5,utilidades_domesticas,6964
6,relogios_presentes,5991
7,telefonia,4545
8,ferramentas_jardim,4347
9,automotivo,4235


### Insight 3: Valor da Compra vs. Parcelamento
Depois de saber *onde* nossos clientes estão (Insight 1) e *o que* eles compram (Insight 2), a próxima pergunta é *como* eles pagam. 

Esta análise investiga se existe uma correlação entre o valor de um produto e a decisão do cliente de parcelar a compra. Para isso, vamos cruzar os dados das tabelas de pagamentos (`payments`), pedidos (`orders`), itens de pedido (`order_items`) e produtos (`products`).

In [5]:
query_pagamentos = """
    SELECT
        p.product_category_name AS categoria,
        AVG(pay.payment_value) AS valor_medio_compra,
        AVG(pay.payment_installments) AS media_de_parcelas,
        COUNT(o.order_id) AS total_de_compras_cartao
    FROM
        payments pay
    JOIN
        orders o ON pay.order_id = o.order_id
    JOIN
        order_items oi ON o.order_id = oi.order_id
    JOIN
        products p ON oi.product_id = p.product_id
    WHERE
        pay.payment_type = 'credit_card' -- Foco apenas em cartão de crédito
        AND p.product_category_name IS NOT NULL
    GROUP BY
        p.product_category_name
    HAVING
        COUNT(o.order_id) > 50 -- Filtra categorias com poucas vendas para ter relevância
    ORDER BY
        valor_medio_compra DESC -- Ordena pelas categorias mais caras
    LIMIT 15;
"""

# Executando a consulta
analise_pagamentos = sqldf(query_pagamentos)

# Exibindo o resultado
print("Análise de Parcelamento por Categoria (Top 15 por Valor Médio)")
display(analise_pagamentos)

Análise de Parcelamento por Categoria (Top 15 por Valor Médio)


,categoria,valor_medio_compra,media_de_parcelas,total_de_compras_cartao
0,pcs,1320.278678,7.333333,174
1,telefonia_fixa,794.757957,3.118280,186
2,portateis_casa_forno_e_cafe,700.644308,6.384615,65
3,eletrodomesticos_2,592.021243,5.497297,185
4,agro_industria_e_comercio,421.888897,4.393103,145
5,moveis_escritorio,392.349062,5.195142,1194
6,instrumentos_musicais,358.830593,4.474308,506
7,construcao_ferramentas_seguranca,358.197655,4.365517,145
8,eletroportateis,337.223513,4.314126,538
9,climatizacao,322.063378,3.720721,222


**Conclusão (Insight 3):**

A análise confirma fortemente a hipótese. Existe uma correlação direta entre o ticket médio de uma categoria e o número médio de parcelas utilizado pelo cliente.

Categorias com um `valor_medio_compra` mais alto, como `pcs` e `relogios_presentes`, apresentam uma `media_de_parcelas` significativamente maior. Isso sugere que a opção de parcelamento é um fator decisivo e estratégico para a conversão de vendas de produtos de maior valor agregado.